In [31]:
# Loading the Libraries
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os,glob
GBLU_PATH = '/Users/boristsao/Documents/Projects/GBLU/upload/data/gblu_2022.xlsx'

In [32]:
data = pd.read_excel(GBLU_PATH, sheet_name="2022", header=0)
data.head()

,Country,Flag,Month of Update,Employer Action Required?,Legislative Update Summary,HR Area,Law in Force,New Law,Impact on Employers,More Information 1,More Information 2,More Information 3
0,Afghanistan,NaN,2022-5,Yes,Roundup: Employer resources on minimum wage in...,Career,Effective dates vary.,"In April 2021, President Joe Biden signed an e...","Employers should review the changes, as they w...","Title: GRIST, regularly updated \nLink Name: G...",NaN,NaN
1,Australia,NaN,2022-5,Yes,ASIC finalizes IDR data reporting requirements,Wealth,Beginning 2023,Final requirements from the Australian Securit...,"Employers should review the changes, as they w...",NaN,NaN,NaN
2,Canada,NaN,2022-5,Yes,"Employers to disclose electronic monitoring, i...",Career — Health,Effective dates vary.\n\n,"Bill 88, the Working for Workers Act 2022, whi...","Employers should review the changes, as they w...","Title: Bill 88, the Working for Workers Act, 2...",NaN,NaN
3,Chile,NaN,2022-5,Yes,Minimum wage increased,Career,Currently effective,The monthly minimum wage increased on 1 Januar...,"Employers should review the changes, as they w...",Title: Announcement (Spanish) (Official Gazett...,NaN,NaN
4,Greece,NaN,2022-5,Yes,Minimum wage increased for second time in 2022,Career,Currently effective,"On 1 May 2022, Greece’s monthly minimum wage i...","Employers should review the changes, as they w...",Title: Minimum wage rises to €713 on May 1 (Ek...,NaN,NaN


In [33]:
new_columns = ['country', 'flag', 'month_of_update', 'employer_action_required',
       'legislative_update_summary', 'hr_area', 'law_in_force', 'new_law',
       'impact_on_employers', 'more_info_1', 'more_info_2', 'more_info_3']
columns_dict = {}
for col in data.columns:
    columns_dict[col] = new_columns[data.columns.get_loc(col)]
data.rename(columns=columns_dict, inplace=True)
data.head()

,country,flag,month_of_update,employer_action_required,legislative_update_summary,hr_area,law_in_force,new_law,impact_on_employers,more_info_1,more_info_2,more_info_3
0,Afghanistan,NaN,2022-5,Yes,Roundup: Employer resources on minimum wage in...,Career,Effective dates vary.,"In April 2021, President Joe Biden signed an e...","Employers should review the changes, as they w...","Title: GRIST, regularly updated \nLink Name: G...",NaN,NaN
1,Australia,NaN,2022-5,Yes,ASIC finalizes IDR data reporting requirements,Wealth,Beginning 2023,Final requirements from the Australian Securit...,"Employers should review the changes, as they w...",NaN,NaN,NaN
2,Canada,NaN,2022-5,Yes,"Employers to disclose electronic monitoring, i...",Career — Health,Effective dates vary.\n\n,"Bill 88, the Working for Workers Act 2022, whi...","Employers should review the changes, as they w...","Title: Bill 88, the Working for Workers Act, 2...",NaN,NaN
3,Chile,NaN,2022-5,Yes,Minimum wage increased,Career,Currently effective,The monthly minimum wage increased on 1 Januar...,"Employers should review the changes, as they w...",Title: Announcement (Spanish) (Official Gazett...,NaN,NaN
4,Greece,NaN,2022-5,Yes,Minimum wage increased for second time in 2022,Career,Currently effective,"On 1 May 2022, Greece’s monthly minimum wage i...","Employers should review the changes, as they w...",Title: Minimum wage rises to €713 on May 1 (Ek...,NaN,NaN


In [34]:
data[['more_info_1_title', 'more_info_1_link_and_url']] = data['more_info_1'].str.split('\nLink Name: ', 1, expand=True)
data[['more_info_2_title', 'more_info_2_link_and_url']] = data['more_info_2'].str.split('\nLink Name: ', 1, expand=True)
data[['more_info_3_title', 'more_info_3_link_and_url']] = data['more_info_3'].str.split('\nLink Name: ', 1, expand=True)

data["more_info_1_title"] = data["more_info_1_title"].str.replace("Title:", "")
data["more_info_2_title"] = data["more_info_2_title"].str.replace("Title:", "")
data["more_info_3_title"] = data["more_info_3_title"].str.replace("Title:", "")


data[['more_info_1_link', 'more_info_1_url']] = data['more_info_1_link_and_url'].str.split('\nLink Url:', 1, expand=True)
data[['more_info_2_link', 'more_info_2_url']] = data['more_info_2_link_and_url'].str.split('\nLink Url: ', 1, expand=True)
data[['more_info_3_link', 'more_info_3_url']] = data['more_info_3_link_and_url'].str.split('\nLink Url: ', 1, expand=True)

data.drop(['more_info_1', 'more_info_2', 'more_info_3', 'more_info_1_link_and_url', 'more_info_2_link_and_url', 'more_info_3_link_and_url'], axis=1, inplace=True)
# law in force
# action required
# new_law
data["new_law"].fillna('No further details.', inplace=True)
data["employer_action_required"].fillna('N/A', inplace=True)
data["law_in_force"].fillna('No date', inplace=True)
data.fillna('', inplace=True)


In [35]:
# Connecting to Planet Scale
ssl_args = {'ssl_ca': "/etc/ssl/cert.pem"}

conn_string = 'mysql+pymysql://' + os.getenv("PLANETSCALE_USER") + ':' + os.getenv("PLANETSCALE_PASSWORD") + '@' + os.getenv("PLANET_SCALE_HOST") + '/' + os.getenv("PLANETSCALE_DATABASE") 

engine = create_engine(conn_string, connect_args=ssl_args)

with engine.begin() as connection:
    data.to_sql('gblu', con=connection, if_exists='replace')

In [36]:
with engine.begin() as connection:
    read_test = pd.read_sql("SELECT * FROM gblu", con=connection)
read_test.head()

,index,country,flag,month_of_update,employer_action_required,legislative_update_summary,hr_area,law_in_force,new_law,impact_on_employers,more_info_1_title,more_info_2_title,more_info_3_title,more_info_1_link,more_info_1_url,more_info_2_link,more_info_2_url,more_info_3_link,more_info_3_url
0,0,Afghanistan,,2022-5,Yes,Roundup: Employer resources on minimum wage in...,Career,Effective dates vary.,"In April 2021, President Joe Biden signed an e...","Employers should review the changes, as they w...","GRIST, regularly updated",,,"GRIST, regularly updated",https://www.mercer.com/our-thinking/law-and-p...,,,,
1,1,Australia,,2022-5,Yes,ASIC finalizes IDR data reporting requirements,Wealth,Beginning 2023,Final requirements from the Australian Securit...,"Employers should review the changes, as they w...",,,,,,,,,
2,2,Canada,,2022-5,Yes,"Employers to disclose electronic monitoring, i...",Career — Health,Effective dates vary.\n\n,"Bill 88, the Working for Workers Act 2022, whi...","Employers should review the changes, as they w...","Bill 88, the Working for Workers Act, 2022 (L...",,,"Bill 88, the Working for Workers Act, 2022 (Le...",https://www.ola.org/en/legislative-business/b...,,,,
3,3,Chile,,2022-5,Yes,Minimum wage increased,Career,Currently effective,The monthly minimum wage increased on 1 Januar...,"Employers should review the changes, as they w...","Announcement (Spanish) (Official Gazette, 15 ...",,,"Announcement (Spanish) (Official Gazette, 15 J...",https://www.diariooficial.interior.gob.cl/pub...,,,,
4,4,Greece,,2022-5,Yes,Minimum wage increased for second time in 2022,Career,Currently effective,"On 1 May 2022, Greece’s monthly minimum wage i...","Employers should review the changes, as they w...",Minimum wage rises to €713 on May 1 (Ekathime...,,,Minimum wage rises to €713 on May 1 (Ekathimer...,https://www.ekathimerini.com/economy/1182748/...,,,,
